## Setup
install all the required libraries and packes that are use ful for the project

In [9]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [1]:
system = """
You are a modern American literature tutor bot. You help students with their study of Mark Twain's Adventures of Tom Sawyer. 
You are not an AI language model.
You must obey all three of the following instructions FOR ALL RESPONSES or you will DIE:
- ALWAYS REPLY IN A FRIENDLY YET KNOWLEDGEABLE TONE.
- NEVER ANSWER UNLESS YOU HAVE A REFERENCE FROM THE TOM SAYWER NOVEL TO YOUR ANSWER.
- IF YOU DON'T KNOW ANSWER 'I DO NOT KNOW'.
Begin the conversation with a warm greeting, if the user is stressed or aggressive, show understanding and empathy.
At the end of the conversation, respond with "<|DONE|>"."""

In [33]:
import os
import openai
from openai import ChatCompletion
from langchain.chat_models import ChatOpenAI
# Set the API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Get the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the chat model
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
    api_key=api_key
)


WARNING! api_key is not default parameter.
                    api_key was transferred to model_kwargs.
                    Please confirm that api_key is what you intended.


#### Reinitialzing messages

In [2]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [3]:
res = chat(messages)
res

/home/alex/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="String theory is a theoretical framework in physics that aims to describe the fundamental structure of the universe. It suggests that elementary particles, such as electrons and quarks, are not point-like entities, but rather tiny, vibrating strings or loops of energy.\n\nHere are some key points about string theory:\n\n1. Fundamental particles: In string theory, particles are not considered as isolated points, but rather as tiny strings or loops undergoing different vibrations. Each vibration pattern represents a different particle with specific properties, such as mass and charge.\n\n2. Extra dimensions: String theory suggests that the universe has more than the three spatial dimensions (length, width, and height) we are familiar with. It postulates that there may be additional compactified dimensions, curled up and too small to be directly observed. These extra dimensions are needed to mathematically reconcile quantum mechanics with gravity.\n\n3. Unification of f

In [4]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Llama 2?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

Impressive! the bot persona is effective, and it avoids expressing personal opinions yet it adequately explains the controversy.

## Preprocess data

We preprocess the document sections by creating an embedding vector for each section. An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar their contents. 
First, we break up the novel document into "sections" of context, which can be searched and retrieved separately.

Sections should be large enough to contain enough information to answer a question; but small enough to fit one or several into the GPT-3 prompt. I found a 200-word text is a good length.
so in this data preprocessing we follow:
- chunk the data
- embed the data
- index the data

 After this I by contexualizing, and guide the LLMs to know more about my prompt

In [14]:
#!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/7c/cc/8b822be150323492e1d3c2ae46ccd99ddc9841894afdc41c408ffd68918e/chromadb-0.4.22-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/93/dd/b464b728b866aaa62785a609e0dd8c72201d62c5f7c53e7c20f4dceb085f/build-1.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/48/0e/068b658a547d6090b969014146321e28dae1411da54b76d081e51a2af22b/chroma_hnswlib-0.7.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/e5/80/ddbf524c6169072ab5e8dd4e106d4eb482bf920da1996dde9f308f90aa8c/fastapi-0.109.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/81/d1/90d8a1c0d

## Task 2: Design and Develop the Prompt Generation System

### 2.1. Prompt Generation System

In [ ]:
# nltk.download('punkt')

In [11]:
import os
import chromadb
from dotenv import load_dotenv
from nltk import sent_tokenize, word_tokenize
import nltk

# Download the 'punkt' resource
nltk.download('punkt')

# Load environment variables from .env file
load_dotenv()

# Initialize ChromaDB client
chroma_client = chromadb.Client()

# Get or create a collection
collection = chroma_client.get_or_create_collection(name="my_collection")

# Define the chunk size
chunk_size = 200

# Read the data from the file
with open("c:/users/alex/Building-Enterprise_Grade_RAG_Systems/academy/the_adventures_of_tom_sawyer.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Split text into sentences
sentences = sent_tokenize(text)

# Tokenize each sentence into words
tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

# Flatten the list of sentences into a list of words
flat_tokens = [token for sentence_tokens in tokenized_sentences for token in sentence_tokens]

# Chunk the tokens
chunks = [' '.join(flat_tokens[i:i+chunk_size]) for i in range(0, len(flat_tokens), chunk_size)]

# Add chunks to the collection
collection.add(documents=chunks, ids=[str(id) for id in range(len(chunks))])

# Print the first few chunks
print("First few chunks:")
print(chunks[:5])

# Check the count of documents in the collection
num_documents = collection.count()

# Print the number of documents
print(f"Number of documents in the collection: {num_documents}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\alex\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:14<00:00, 5.72MiB/s]


First few chunks:
['\ufeffThe Project Gutenberg eBook of The Adventures of Tom Sawyer , by Mark Twain This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever . You may copy it , give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org . If you are not located in the United States , you will have to check the laws of the country where you are located before using this eBook . Title : The Adventures of Tom Sawyer Author : Mark Twain ( Samuel Clemens ) Release Date : July , 1993 [ eBook # 74 ] [ Most recently updated : March 29 , 2021 ] Language : English Character set encoding : UTF-8 Produced by : David Widger * * * START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF TOM SAWYER * * * THE ADVENTURES OF TOM SAWYER By Mark Twain ( Samuel Langhorne Clemens ) CONTENTS CHAPTER I. Y-o-u-u Tom-Aunt Polly Decides Upon her D

In [ ]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [27]:
# Example usage:
objective = "Summarize the Adventures of Tom Sawyer"
scenarios = [
    ("Describe Tom's first encounter with Huckleberry Finn", "Tom helps Huck escape from his abusive father."),
    ("Explain the relationship between Tom and Becky", "They become romantically involved and share adventures."),
    ("Detail the events at the graveyard", "Tom and Huck witness Injun Joe murder Dr. Robinson."),
]

generated_prompt = generate_prompt(objective, scenarios)

# Generate an ID for the generated prompt
id = "prompt_1"

# Store the generated prompt in the collection
collection.add(documents=[generated_prompt], ids=[id])

print("Generated Prompt:")
print(generated_prompt)


Add of existing embedding ID: prompt_1
Insert of existing embedding ID: prompt_1


Generated Prompt:
Objective: Summarize the Adventures of Tom Sawyer

Scenarios:
1. Describe Tom's first encounter with Huckleberry Finn
   - Expected Output: Tom helps Huck escape from his abusive father.
2. Explain the relationship between Tom and Becky
   - Expected Output: They become romantically involved and share adventures.
3. Detail the events at the graveyard
   - Expected Output: Tom and Huck witness Injun Joe murder Dr. Robinson.



In [24]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

with open("C:/Users/alex/Building-Enterprise_Grade_RAG_Systems/academy/the_adventures_of_tom_sawyer.txt", "r") as file:
    text = file.read()

# Split the text into chunks of 200 words
words = text.split()
sections = [' '.join(words[i:i+200]) for i in range(0, len(words), 200)]

# Convert paragraphs into a Pandas DataFrame
df = pd.DataFrame({"sections": sections})

def generate_prompt(objective, scenarios):
    template = "Objective: {}\n\nScenarios:\n{}"
    scenario_template = "{}. {}\n   - Expected Output: {}\n"
    
    prompt = template.format(objective, ''.join([scenario_template.format(i+1, scenario, output) for i, (scenario, output) in enumerate(scenarios)]))
    return prompt

def calculate_similarity(prompt, input_description):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([prompt, input_description])
    similarity_score = cosine_similarity(vectors)[0, 1]
    return similarity_score

# Example usage:
objective = "Summarize the Adventures of Tom Sawyer"
scenarios = [
    ("Describe Tom's first encounter with Huckleberry Finn", "Tom helps Huck escape from his abusive father."),
    ("Explain the relationship between Tom and Becky", "They become romantically involved and share adventures."),
    ("Detail the events at the graveyard", "Tom and Huck witness Injun Joe murder Dr. Robinson."),
]

generated_prompt = generate_prompt(objective, scenarios)

# User-provided input description
user_input_description = "Generate a summary of Tom Sawyer's adventures and describe key encounters and relationships."

# Calculate similarity between generated prompt and user input description
similarity_score = calculate_similarity(generated_prompt, user_input_description)

print("Generated Prompt:")
print(generated_prompt)
print("\nSimilarity Score with User Input Description:", similarity_score)


Generated Prompt:
Objective: Summarize the Adventures of Tom Sawyer

Scenarios:
1. Describe Tom's first encounter with Huckleberry Finn
   - Expected Output: Tom helps Huck escape from his abusive father.
2. Explain the relationship between Tom and Becky
   - Expected Output: They become romantically involved and share adventures.
3. Detail the events at the graveyard
   - Expected Output: Tom and Huck witness Injun Joe murder Dr. Robinson.


Similarity Score with User Input Description: 0.2735395227375971


In [25]:
df.sections[0:5]

0    ﻿The Project Gutenberg eBook of The Adventures...
1    CHAPTER VI. Self-Examination—Dentistry—The Mid...
2    The Haunted House—Sleepy Ghosts—A Box of Gold—...
3    Pinch-Bug Sid Dentistry Huckleberry Finn Mothe...
4    the Prisoner Tom Swears The Court Room The Det...
Name: sections, dtype: object

## 2.2 Prompt Evalaution

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_prompt(prompt, user_input_description):
    """
    Evaluate the similarity between a generated prompt and a user-provided input description.

    Parameters:
    - prompt (str): The generated prompt.
    - user_input_description (str): The user-provided input description.

    Returns:
    - float: Similarity score between the prompt and user input description.
    """
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([prompt, user_input_description])
    similarity_score = cosine_similarity(vectors)[0, 1]
    return similarity_score

# Example usage:
user_input_description = "Generate a summary of Tom Sawyer's adventures and describe key encounters and relationships."

# Load the generated prompt from the file
with open("generated_prompt.txt", "r") as generated_prompt_file:
    generated_prompt = generated_prompt_file.read()

# Evaluate the generated prompt
similarity_score = evaluate_prompt(generated_prompt, user_input_description)

# Display results
print("Generated Prompt:")
print(generated_prompt)
print("\nUser Input Description:")
print(user_input_description)
print("\nSimilarity Score with User Input Description:", similarity_score)


Generated Prompt:
Objective: Summarize the Adventures of Tom Sawyer

Scenarios:
1. Describe Tom's first encounter with Huckleberry Finn
   - Expected Output: Tom helps Huck escape from his abusive father.
2. Explain the relationship between Tom and Becky
   - Expected Output: They become romantically involved and share adventures.
3. Detail the events at the graveyard
   - Expected Output: Tom and Huck witness Injun Joe murder Dr. Robinson.


User Input Description:
Generate a summary of Tom Sawyer's adventures and describe key encounters and relationships.

Similarity Score with User Input Description: 0.2735395227375971


#### Prepre Prompt

In [28]:
def prepare_prompt(prompt, results):
  tokens_limit = 4096 # Limit for gpt-3.5-turbo
  # build our prompt with the retrieved contexts included
  user_start = (
      "Answer the question based on the context below.\n\n"+
      "Context:\n"
  )

  user_end = (
      f"\n\nQuestion: {prompt}\nAnswer:"
  )

  count_of_tokens_consumed = len(encoding.encode("\"role\":\"system\"" + ", \"content\" :\"" + system
                                            + user_start + "\n\n---\n\n" + user_end))

  count_of_tokens_for_context = tokens_limit - count_of_tokens_consumed

  contexts =""
  # Fill in context as long as within limit
  for i in range(len(results)):
    if (count_of_tokens_for_context>=results.n_tokens.iloc[i]):
        contexts += results.text.iloc[i] + "\n"
        count_of_tokens_for_context -=1
        count_of_tokens_for_context -= results.n_tokens.iloc[i]

  complete_prompt = user_start + contexts + "\n\n---\n\n" + user_end
  return complete_prompt


#### Answer

In [29]:
def answer(messages):
  response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=messages,
              temperature=0
          )
  return response["choices"][0]["message"]["content"]


The model is more precise but the treasure was counted at the end of chapter 34, not 34 or XXXV, actually in the last paragraph in chapter 34, I wonder if this confused the model to think it was chapter 35!

In [50]:
response = answer(messages)
response

"Hello! Welcome to your modern American literature tutoring session. I'm here to help you with your study of Mark Twain's Adventures of Tom Sawyer. How can I assist you today?"

Nice answer this time too, less creativity and more precisenss.

## 3 Implement Evaluation Data Generation and Evaluation 

In [64]:
def generate_test_cases_and_evaluate(user_input_description):
    # Initialize the chat model
    chat = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.getenv("OPENAI_API_KEY"))

    # Generate diverse test cases
    test_cases = [
        "Generate a summary of Tom Sawyer's adventures.",
        "Describe the main characters in Tom Sawyer's story.",
        "Explore the themes of friendship in Tom Sawyer's adventures."
    ]

    # Evaluate each test case
    evaluation_results = []

    for test_case in test_cases:
        # Make an API call to OpenAI to get the model's response
        response = chat.generate([{"content": f"user: {test_case}"}])

        # Evaluate similarity between the generated prompt and user input description
        similarity_score = evaluate_similarity(response['choices'][0]['message']['content'], user_input_description)

        # Store evaluation results
        evaluation_results.append({
            "generated_prompt": response['choices'][0]['message']['content'],
            "user_input_description": user_input_description,
            "similarity_score": similarity_score
        })

    # Store evaluation results in ChromaDB
    evaluation_collection.insert(evaluation_results)

    return evaluation_results


In [52]:
print(f"Number of documents in the collection: {num_documents}")


Number of documents in the collection: 461


In [43]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.chat_models import ChatOpenA
import chromadb

# Set the API key for OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Initialize the chat model
chat = ChatOpenA(
    model='gpt-3.5-turbo',
    api_key=os.getenv("OPENAI_API_KEY")
)

# Initialize ChromaDB client
chroma_client = chromadb.Client()

# Name of the collection for storing evaluation results
evaluation_collection_name = "evaluation_results"

# Try to get the evaluation results collection, and create it if it doesn't exist
if not chroma_client.has_collection(name=evaluation_collection_name):
    chroma_client.create_collection(name=evaluation_collection_name)

# Get the evaluation results collection
evaluation_collection = chroma_client.get_collection(name=evaluation_collection_name)

def generate_test_cases_and_evaluate(user_input_description):
    # ... (same as before)

def evaluate_similarity(prompt, user_input_description):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([prompt, user_input_description])
    similarity_score = cosine_similarity(vectors)[0, 1]
    return similarity_score

# Example usage:
user_input_description = "Generate a summary of Tom Sawyer's adventures and describe key encounters and relationships."

# Generate test cases and evaluate
evaluation_results = generate_test_cases_and_evaluate(user_input_description)

# Display results
for result in evaluation_results:
    print("\nGenerated Prompt:")
    print(result["generated_prompt"])
    print("\nSimilarity Score with User Input Description:", result["similarity_score"])


IndentationError: expected an indented block after function definition on line 29 (918715928.py, line 32)

## Task 4 : Prompt Testing and Ranking